# Virtual Scribe AI Comparator

The goal of this notebook is to compare the ability of various AI models to generate a Geriatrics Consultation Note

In [1]:
# Load necessary components
import os

from langchain.llms import Ollama
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate


In [ ]:
# Pick models to test
models = ["llama3.2", "llama3", "qwen2"]

# Get notes
notes = [
    """Alice Wonderland 84 year old female
    Has headache
    history of diabetes, multiple falls
    Recent fall 3 days ago prior to onset of headache
    Has been getting worse
    Recommend she go to the ED to rule out intracranial pathology
    Her GDS is 15/15
    He has no history of smoking or tobacco use
    """,
]

In [5]:
# Create the prompt
prompt = PromptTemplate(
    input_variables=["notes"],
    template="""You are a virtual medical scribe who needs to write a Geriatric consultation note. Use the following template as the output to fill the information provided. For any missing information, please write '***NEEDS TO BE COLLECTED***'

    Patient Name:
    Date of Birth:

    Chief Complaint:

    History of Presenting Illness:

    Personal Medical History:
    Surgical History:
    Social History:
    Family History:
    
    COMPREHENSIVE GERIATRIC ASSESSMENT:
    Matters Most:
        What matters most to patient?
    Mobility:
        History of Falls?
    Medications:
        Does patient have polypharmacy?
    Mind:
        Geriatric Depression Scale:
        SLUMS Test:
    Multicomplexity:
        Summary of what makes treatment of patient challenging
    
        
    ASSESSMENT AND PLAN:

    That is the template to follow. Use the following information to fill out the above template: {notes}
    """
)

In [ ]:
for model in models:
    llm = Ollama(model=model)
    for note in notes:
        print(LLMChain(llm=llm,prompt=prompt).invoke(note))

{'notes': 'Alice Wonderland 84 year old female\n    Has headache\n    history of diabetes, multiple falls\n    Recent fall 3 days ago prior to onset of headache\n    Has been getting worse\n    Recommend she go to the ED to rule out intracranial pathology\n    Her GDS is 15/15\n    He has no history of smoking or tobacco use\n    ', 'text': "Patient Name: Alice Wonderland\n\nDate of Birth: ***NEEDS TO BE COLLECTED***\n\nChief Complaint: Headache\n\nHistory of Presenting Illness:\nThe patient reports a worsening headache over the past few days, with a recent fall 3 days ago prior to the onset of symptoms.\n\nPersonal Medical History:\n- Diabetes\n***NEEDS TO BE COLLECTED***\n\nSurgical History:\n- ***NEEDS TO BE COLLECTED***\nNo specific information provided\n\nSocial History:\n- None mentioned\nNo information about the patient's social history is available\n\nFamily History:\n- ***NEEDS TO BE COLLECTED***\nNo family history was provided\n\nCOMPREHENSIVE GERIATRIC ASSESSMENT:\n\nMatters